In [ ]:
# Set 1
# Purpose: Help kids to find the optimal path to their friends houses during Halloween in a neighborhood (trick or treat) ^.^
# By implementing of Held-Karp, an algorithm that solves the Traveling Salesman Problem using dynamic programming with memoization

import itertools
import random
import sys

def held_karp(dists): # dists: distance matrix
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}

    # Set transition cost from initial state(StartNode)(kid's own house which is 0)
    # C[A:distance]
      # A = (bit indicator, CurrentNode(the house that current go))
        # bit indicator as (10 in base 2) is the 2nd house, (100 in base 2) is the 3rd house . . .
          # in decimal '2' = 2nd house, and '4' = 3rd house ...
      # distance = (Distance between PreviousNode to CurrentNode , PreviousNode)
        # Distance between PreviousNode to CurrentNode = (distance between house that kid from to the current going house)
        # PreviousNode = (the previous visited house that kid from)
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset_size in range(2, n): # looping the the subset to by increasing the subset_size(how many house choose to go)
        for subset in itertools.combinations(range(1, n), subset_size):
            #itertools.combinations(range(1, n), subset_size) will return all combination of houses to go
            #Based on the (range(1, n)(total houses - 1) ,subset_size(how many house choose to go))
              #(range(1, n)(total houses - 1) minus 1 to exlcude the kid's own house)

            # Set bits for all nodes in this subset
            bits = 0
            for bit in subset:
                bits |= 1 << bit # Marking down 'all the houses choosing to go'(the subset that is combination of houses to go) with bits indicator

            # Find the lowest cost to get to this subset
            # Find the minimum distance in terms of going all the house chooosen in the subset
            for k in subset:
                prev = bits & ~(1 << k) # Turning off the bit = (Based 2 value '1' -> Based 2 value '0') for house act as the last house to visit(EndNode)
                # ~ is NOT operator
                res = []
                for m in subset:
                    if m == 0 or m == k: # The 'current house'(CurrentNode) to find distance cannot be the 'last house to visited'(EndNode)
                                         # as well as the 'kid's own house'(StartNode)
                        continue # Going to the next house if 'current house'(CurrentNode) = (EndNode) or (StartNode)
                    res.append((C[(prev, m)][0] + dists[m][k], m)) # Adding the two distances
                                                                   # distance for (OtherNode) to the (EndNode)
                                                                    # (OtherNode) = all other node without including (EndNode) as well as (StartNode)
                                                                     # The choosen houses not including kid's own house and the last house to visit
                                                                   # distance for (CurrentNode) to the (EndNode)
                C[(bits, k)] = min(res) # Return the optimal/minimum total path distance with the last visited house(Endnode).

    # We're interested in all bits but the least significant (the start state)
    # Which means we will not take the StartNode (kid's houses) into the bits indicator
    bits = (2**n - 1) - 1

    # Calculate optimal cost
    # Take the optimal/minimum total distance path for going home(StartNode) as assumeing very house is the last visited house(Endnode)
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, parent = min(res) # Return the optimal/minimum total path distance with the last visited house(Endnode).
    # Which is the house the kid go before going home.

    # Backtrack to find full path
    path = []
    for i in range(n - 1):
        path.append(parent) # Put the last visited house(Endnode) that we find just now
        new_bits = bits & ~(1 << parent) # Backtracking the previousNode(previous visited house) using bit indicator by 'turning off the bit'
                                         # Turning off the bit = (Based 2 value '1' -> Based 2 value '0')
                                         # The 'bit is still on'(Based 2 value = '1') are/is the house(s) left to backtracking
        _, parent = C[(bits, parent)] # Take the previousNode. Which is (the previous visited house that kid from)
        bits = new_bits # Assign the new bit indicator to continue backtracking

    # Add implicit start state to indicate starting house is kid's house(StartNode)
    path.append(0)

    # Return result which is a tuple, (cost, pathAarry[]).
    result = [opt, list(reversed(path))] # We need to use reversed function to reversed path since the path is backtrack just now
    return result


def generate_distances(n): # Random generate distane for kids' friends' houses
    dists = [[0] * n for i in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            dists[i][j] = dists[j][i] = random.randint(1, 10)

    # You can also set the distances by yourself with 2d array like below with three houses (including kid's house)
    # dists = [[0,5,7],[5,0,6],[7,6,0]]
    return dists
# Can change the number of friend houses for kid to go
n = 4

dists = generate_distances(n)

# Pretty-print the distance matrix
for row in dists:
  print(''.join([str(n).rjust(3, ' ') for n in row]))
print('')
print("For this example is there are " + str(n) +" friend houses")
print("The optimal distance for kid is " + str(held_karp(dists)[0]) + " and travel path is " + str(held_karp(dists)[1]))
print("Assume 0 is the house of kid that are intend to going out")
for x in range(len(held_karp(dists)[1])-1):
  if (x == 0):
    continue
  print("Then will follow the path going to house " + str(held_karp(dists)[1][x]))
print("At the end going back to home from house " + str(held_karp(dists)[1][len(held_karp(dists)[1])-1]))

  0  1  6  7
  1  0  2  5
  6  2  0  8
  7  5  8  0

For this example is there are 4 friend houses
The optimal distance for kid is 18 and travel path is [0, 3, 2, 1]
Assume 0 is the house of kid that are intend to going out
Then will follow the path going to house 3
Then will follow the path going to house 2
At the end going back to home from house 1
